In [ ]:
import time

from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.enums import ObjectType
from pycram.failure_handling import Retry, RetryMonitor
from pycram.fluent import Fluent
from pycram.language import Monitor, Code
from pycram.plan_failures import NotALanguageExpression
from pycram.pose import Pose
from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.ros.viz_marker_publisher import VizMarkerPublisher

In [ ]:
world = BulletWorld("DIRECT")
viz = VizMarkerPublisher()
robot = Object("pr2", "robot", "../../resources/pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", "environment", "../../resources/apartment.urdf")

milk = Object("milk", "milk", "../../resources/milk.stl", pose=Pose([2.5, 2, 1.02]), color=[0, 0, 1, 1])
cereal = Object("cereal", "cereal", "../../resources/breakfast_cereal.stl", pose=Pose([2.5, 2.3, 1.05]),
                color=[1, 0, 0, 1])

pick_pose = Pose([2.7, 2.15, 1])

robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

obj_type = "milk"
obj_color = "blue"
obj_name = ""
obj_location = ""
obj_size = "Normal"


def color_map(color):
    color_switch = {
        "red": [1, 0, 0, 1],
        "green": [0, 1, 0, 1],
        "blue": [0, 0, 1, 1],
        "black": [0, 0, 0, 1],
        "white": [1, 1, 1, 1],
        # add more colors if needed
    }
    color = color_switch.get(color)
    if color is None:
        color = [0, 0, 0, 1]
    return color


def monitor_func():
    if not hasattr(monitor_func, 'tries_counter'):
        monitor_func.tries_counter = 0
    else:
        monitor_func.tries_counter += 1

    if monitor_func.tries_counter == 0:
        return NotALanguageExpression
    elif monitor_func.tries_counter == 1:
        return PerceptionObjectNotFound
    else:
        return False


@with_simulated_robot
def move_and_detect(obj_type, obj_size, obj_color):
    NavigateAction(target_locations=[Pose([1.7, 2, 0])]).resolve().perform()

    LookAtAction(targets=[pick_pose]).resolve().perform()
    status, object_dict = DetectAction(technique='all').resolve().perform()
    object_desig = object_dict[obj_type]

    return object_desig


def announce_pick(name: str, type: str, color: str, location: str, size: str):
    print(f"I will now perceive up the {size.lower()} {color.lower()} {type.lower()} at location {location.lower()} ")
    time.sleep(5)


def announce_recovery1():
    print("Recovery plan 1")
    time.sleep(1)
    MoveTorsoAction([0.01]).resolve().perform()
    time.sleep(1)
    MoveTorsoAction([0.25]).resolve().perform()


def announce_recovery2():
    print("Recovery plan 2")
    time.sleep(1)
    MoveTorsoAction([0.3]).resolve().perform()
    time.sleep(1)
    MoveTorsoAction([0.25]).resolve().perform()


In [ ]:
with simulated_robot:
    ###### Prepare robot ######
    ParkArmsAction.Action(Arms.BOTH).perform()

    MoveTorsoAction([0.25]).resolve().perform()

    ###### Announce object and wait ######
    announce = Code(lambda: announce_pick(obj_name, obj_type, obj_color, obj_location, obj_size))

    ###### Move and detect object ######
    milk_desig = Code(lambda: move_and_detect(obj_type, obj_size, obj_color))

    ###### construct and monitor subplan ######
    plan = announce + milk_desig >> Monitor(monitor_func)

    ###### Construct recovery behaviour (Navigate to island => place object => detect new object => pick up new object ######
    recover1 = Code(lambda: announce_recovery1())
    recover2 = Code(lambda: announce_recovery2())

    ###### Execute plan ######
    _, [_, milk_desig] = RetryMonitor(plan, max_tries=5, recovery={NotALanguageExpression: recover1,
                                                                   PerceptionObjectNotFound: recover2}).perform()

    print("Succeeded")